In [1]:
import pandas as pd
import plotly.express as px
from pathlib import Path
import numpy as np
from scipy.stats import f_oneway
from scipy.stats import pearsonr

In [3]:

car_data = pd.read_csv('vehicles.csv')
car_data

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51520,9249,2013.0,nissan maxima,like new,6.0,gas,88136.0,automatic,sedan,black,NaN,2018-10-03,37
51521,2700,2002.0,honda civic,salvage,4.0,gas,181500.0,automatic,sedan,white,NaN,2018-11-14,22
51522,3950,2009.0,hyundai sonata,excellent,4.0,gas,128000.0,automatic,sedan,blue,NaN,2018-11-15,32
51523,7455,2013.0,toyota corolla,good,4.0,gas,139573.0,automatic,sedan,black,NaN,2018-07-02,71


In [4]:
(car_data
 .isna()
.mean()
.sort_values(ascending=False)
.reset_index()
.rename(columns={'index':'Coluna', 0 : 'Missing Data'})
)

,Coluna,Missing Data
0,is_4wd,0.503697
1,paint_color,0.179854
2,odometer,0.153168
3,cylinders,0.102086
4,model_year,0.070238
5,price,0.000000
6,model,0.000000
7,condition,0.000000
8,fuel,0.000000
9,transmission,0.000000


In [5]:
car_data = car_data.dropna()
car_data['model_year'] = car_data['model_year'].astype(str)
car_data['model_year'].dtype

<ipython-input-5-71621aede145>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  car_data['model_year'] = car_data['model_year'].astype(str)


dtype('O')

In [121]:
car_data['brand'] = car_data['model'].apply(lambda x: 'dodge ram' if 'dodge' in x.lower() or 'ram' in x.lower() else x.split()[0])
car_data

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,brand
26808,6000,1965.0,ford f-250,good,6.0,gas,71000.0,manual,pickup,blue,1.0,2018-11-10,22,ford
6096,5000,1975.0,jeep wrangler,good,8.0,gas,100000.0,manual,offroad,grey,1.0,2019-02-01,57,jeep
47073,15895,1977.0,jeep cherokee,good,8.0,gas,23836.0,automatic,SUV,grey,1.0,2018-10-28,31,jeep
50169,7500,1977.0,ford f-250,good,8.0,gas,112000.0,automatic,truck,yellow,1.0,2018-07-03,99,ford
35541,6850,1979.0,ford ranger,excellent,8.0,gas,56000.0,manual,pickup,white,1.0,2018-07-03,51,ford
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20460,30029,2019.0,jeep grand cherokee,like new,6.0,gas,32500.0,automatic,SUV,silver,1.0,2019-03-12,89,jeep
50373,22900,2019.0,nissan frontier crew cab sv,good,6.0,gas,24402.0,other,pickup,blue,1.0,2019-01-06,17,nissan
2037,34990,2019.0,gmc sierra 1500,good,8.0,gas,16989.0,automatic,truck,blue,1.0,2019-04-08,14,gmc
9279,46267,2019.0,ford f-250,like new,8.0,diesel,1440.0,automatic,truck,grey,1.0,2019-02-03,21,ford


In [122]:
car_data = car_data .sort_values(by='model_year')
car_data

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,brand
26808,6000,1965.0,ford f-250,good,6.0,gas,71000.0,manual,pickup,blue,1.0,2018-11-10,22,ford
6096,5000,1975.0,jeep wrangler,good,8.0,gas,100000.0,manual,offroad,grey,1.0,2019-02-01,57,jeep
47073,15895,1977.0,jeep cherokee,good,8.0,gas,23836.0,automatic,SUV,grey,1.0,2018-10-28,31,jeep
50169,7500,1977.0,ford f-250,good,8.0,gas,112000.0,automatic,truck,yellow,1.0,2018-07-03,99,ford
35541,6850,1979.0,ford ranger,excellent,8.0,gas,56000.0,manual,pickup,white,1.0,2018-07-03,51,ford
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26499,4500,2019.0,jeep wrangler,good,6.0,gas,157000.0,manual,SUV,orange,1.0,2019-01-10,12,jeep
18984,39900,2019.0,ford f-250 super duty,excellent,8.0,gas,15304.0,automatic,truck,white,1.0,2019-02-05,61,ford
39600,34100,2019.0,ford f-150,like new,6.0,gas,19000.0,automatic,pickup,silver,1.0,2019-03-25,69,ford
26546,1,2019.0,nissan rogue,excellent,10.0,gas,6952.0,other,SUV,blue,1.0,2019-02-25,55,nissan


In [123]:
def anova(df,column,x_f,x_p,x_axis):
    if df[column].dtype == 'object':
        groups = [df[df[column] == col]['price'] for col in df[column].unique()]
        f_statistic, p_value = f_oneway(*groups)
    else:
        data1 = df[column]
        data2 = df['price']
        f_statistic, p_value = f_oneway(data1, data2)

        print("F-statistic:", f_statistic)
        print("p-value:", p_value)

    if p_value < 0.05:
            print(f"Reject null hypothesis: There are significant differences in mean prices between at least two {column}s.")
    else:
            print(f"Fail to reject null hypothesis: There are no significant differences in mean prices between {column}s.")

    fig = px.scatter(df, x=column, y='price', title=f'Price Distribution by {column}')
    fig.update_layout(xaxis=dict(dtick=2))
    fig.update_xaxes(tickvals=x_axis)
    fig.add_annotation(x=x_f, y=350000, text=f'F-statistic: {f_statistic:.2f}', showarrow=False)
    fig.add_annotation(x=x_p, y=350000, text=f'p-value: {p_value:.2f}', showarrow=False)
    return fig


In [124]:
y_odometer = [i*100000 for i in range(10)]
fig1 = anova(car_data, 'odometer', 400000,700000,y_odometer)
fig1.show()

F-statistic: 35026.938626828705
p-value: 0.0
Reject null hypothesis: There are significant differences in mean prices between at least two odometers.


In [125]:
y_model_year = [i*1 for i in range(45)]
fig2 = anova(car_data, 'model_year', 20,30,y_model_year)
fig2.show()

Reject null hypothesis: There are significant differences in mean prices between at least two model_years.


In [126]:
y_model = [i*2 for i in range(100)]
fig3 = anova(car_data, 'model', 50,70,y_model )
fig3.show()

Reject null hypothesis: There are significant differences in mean prices between at least two models.


In [127]:
y_condition= [i*1 for i in range(6)]
fig4 = anova(car_data, 'condition', 2,4,y_condition)
fig4.show()

Reject null hypothesis: There are significant differences in mean prices between at least two conditions.


In [128]:
y_cylinders= [i*2 for i in range(10)]
fig5 = anova(car_data, 'cylinders', 7,10,y_cylinders)
fig5.show()

F-statistic: 27274.61612513274
p-value: 0.0
Reject null hypothesis: There are significant differences in mean prices between at least two cylinderss.


In [129]:
y_fuel= [i*1 for i in range(5)]
fig6 = anova(car_data, 'fuel', 2,3,y_fuel)
fig6.show()

Reject null hypothesis: There are significant differences in mean prices between at least two fuels.


In [130]:
y_transmission= [i*1 for i in range(3)]
fig7 = anova(car_data, 'transmission', 1,1.5,y_transmission)
fig7.show()

Reject null hypothesis: There are significant differences in mean prices between at least two transmissions.


In [131]:
y_type= [i*1 for i in range(12)]
fig8 = anova(car_data, 'type', 5,8,y_type)
fig8.show()

Reject null hypothesis: There are significant differences in mean prices between at least two types.


In [132]:
y_paint_color= [i*1 for i in range(12)]
fig9 = anova(car_data, 'paint_color', 5,8,y_paint_color)
fig9.show()

Reject null hypothesis: There are significant differences in mean prices between at least two paint_colors.


In [133]:
y_brand= [i*1 for i in range(18)]
fig10 = anova(car_data, 'brand', 5,10,y_brand)
fig10.show()

Reject null hypothesis: There are significant differences in mean prices between at least two brands.


In [134]:
def plot_hist(df):
    brands = df['brand'].unique()
    histograms = []
    for brand in brands:
        brand_separate = df[df['brand']== brand]
        histogram = px.histogram(brand_separate, x='price', title=f'Histogram of Price for {brand}')
        histograms.append(histogram)
    return histograms

In [135]:
histograma = plot_hist(car_data)
for histogram in histograma:
    histogram.show()